In [1]:
from orphics import maps,io # helper functions
from pixell import enmap, curvedsky as cs
import numpy as np
import os,sys
from soapack import interfaces as sints
from falafel import qe,utils as futils
import solenspipe

ModuleNotFoundError: No module named 'actsims'

In [ ]:
# ACT data
# qid = 'd56_01' # D56 S14 PA1 150 GHz, see soapack/soapack/data/all_arrays_dr5.csv
# mask = sints.get_binary_apodized_mask(qid)
# dm = sints.DR5(region=mask)
# imap = dm.get_splits(qid)[0,0]
# dfact = 4
# mask = enmap.downgrade(mask,dfact)
# imap = enmap.downgrade(imap,dfact)

In [ ]:
#Get Data
inkappa_path = '/scratch/r/rbond/msyriac/data/sims/alex/v0.4/'
mask_path = '/home/r/rbond/jiaqu/scratch/DR6/maps/sims/map_masks/BN_bottomcutsmooth1.fits'
mask_map = enmap.read_map(mask_path)

In [ ]:
# Set up lensing reconstruction parameters
shape,wcs = mask.shape,mask.wcs
mlmax = 4000

# beam_fwhm = 1.4
# noise_t = 30.0
beam_fwhm = 0.
noise_t = 0.
ucls, tcls = futils.get_theory_dicts_white_noise(beam_fwhm,noise_t)

In [ ]:
# fTalm  -- beam deconvolved, inverse-variance filtered map
alm = cs.map2alm(imap*mask,lmax=mlmax) # alm transform the map
"""
alm / (C_TT + N_TT/beam**2)  / beam
tcls['TT'] = C_TT + N_TT/beam**2

"""
lmin = 600
lmax = 3000
fTalm = futils.isotropic_filter([alm,alm*0,alm*0],tcls,lmin,lmax,ignore_te=True)[0]
# fTalm = cs.almxfl(fTalm,lambda ells: 1./maps.gauss_beam(ells,beam_fwhm))

# Set up internal geometry
px = qe.pixelization(shape=shape,wcs=wcs)

In [ ]:
#Run the estimator
wrapper = False
if wrapper:
    # Use the SO-lenspipe wrapper
    qfunc = solenspipe.get_qfunc(px,ucls,mlmax,"TT")
    recon_alms = qfunc([fTalm,fTalm*0,fTalm*0],[fTalm,fTalm*0,fTalm*0])
else:
    # Direct call to falafel, more flexible
    recon_alms = qe.qe_all(px,ucls,mlmax,
                           fTalm=fTalm,fEalm=fTalm*0,fBalm=fTalm*0,
                           estimators=['mv'])['mv']

In [ ]:
# Project reconstruction alms to a real-space map
kappa_map = cs.alm2map(recon_alms,enmap.empty(shape,wcs,dtype=np.float32))

# Plot the kappa map
io.hplot(kappa_map,'kappa_map',downgrade=4)